In [3]:
lst = [1,2,2,2,3,4,4,4,5,6,7]
dup = {}

for item in lst:
  if item in dup:
    dup[item] += 1
  else:
    dup[item] = 1
for key in dup:
  # if dup[key] > 1:
    print(key, 'is',dup[key] )

1 is 1
2 is 3
3 is 1
4 is 3
5 is 1
6 is 1
7 is 1


In [4]:
words = ['qqwsw','eerdece','e','frgfrg']
resut = ''
for word in words:
  resut += word + str(len(word))
print(resut)

qqwsw5eerdece7e1frgfrg6


In [18]:
num = 121
strnum = str(num)
trnum = strnum[::-1]

if trnum == strnum:
  print('yes')
else:
  print('no')

yes


In [20]:
num = 121
strnum = str(num)
revnum = strnum[::-1]

if strnum == revnum:
    print('yes')
else:
    print('no')

yes


In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
# 1. Sample Data
data = [
    ("U001", "2024-01-01 10:00:00", 100),
    ("U001", "2024-02-01 09:00:00", 105),
    ("U002", "2024-01-05 11:00:00", 200),
    ("U002", "2024-01-05 08:00:00", 195),
]
columns = ["user_id", "last_updated", "value"]
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(data, columns)
# 2. Convert to timestamp if needed
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp
df = df.withColumn("last_updated", to_timestamp("last_updated"))
# 3. Define Window Spec
window_spec = Window.partitionBy("user_id").orderBy(col("last_updated").desc())
# 4. Assign Row Numbers
ranked_df = df.withColumn("row_num", row_number().over(window_spec))
# 5. Filter for Most Recent Record
final_df = ranked_df.filter(col("row_num") == 1).drop("row_num")
final_df.show()

+-------+-------------------+-----+
|user_id|       last_updated|value|
+-------+-------------------+-----+
|   U001|2024-02-01 09:00:00|  105|
|   U002|2024-01-05 11:00:00|  200|
+-------+-------------------+-----+

